In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from datetime import time as datetime_time
plt.style.use('seaborn-whitegrid')

In [2]:
# = pd.DataFrame(columns=['date', 'washers', 'dryers'])
#raw_data.read_csv("./both", header=None)

file = "./laundry_stats"

headers = ['datetime', 'wash', 'dry']
dtypes = {'datetime': 'str', 'wash': 'int', 'dry': 'int'}
parse_dates = ['datetime']
raw_data = pd.read_csv(file, sep=',', header=None, names=headers, dtype=dtypes, parse_dates=parse_dates, date_parser=pd.to_datetime)
#print(raw_data.dtypes)
print(raw_data)

                  datetime  wash  dry
0      2018-12-16 21:40:05     0    2
1      2018-12-16 21:45:05     0    2
2      2018-12-16 21:50:05     0    2
3      2018-12-16 21:55:06     1    2
4      2018-12-16 22:00:05     1    2
...                    ...   ...  ...
251710 2022-02-06 14:55:11     1    3
251711 2022-02-06 15:00:08     1    2
251712 2022-02-06 15:05:07     1    2
251713 2022-02-06 15:10:07     0    0
251714 2022-02-06 15:15:08     0    0

[251715 rows x 3 columns]


In [3]:
def shift_forward(data):
    local_data=data.copy()
    local_data['datetime'] = [time + timedelta(hours=4) for time in local_data['datetime']]
    return local_data

def shift_back(data):
    local_data=data.copy()
    local_data['datetime'] = [time + timedelta(hours=-4) for time in local_data['datetime']]
    return local_data
    
shifted_data = shift_forward(shift_back(raw_data))
#print(shifted_data)

In [4]:
def get_by_months(data):
    return[g.reset_index() for n, g in data.set_index('datetime').groupby(pd.Grouper(freq='M'))]

def get_by_year(data):
    return[g.reset_index() for n, g in data.set_index('datetime').groupby(pd.Grouper(freq='Y'))]

months = get_by_months(raw_data)
get_by_year(raw_data)
#print(pd.concat(months[1:3]))

[                datetime  wash  dry
 0    2018-12-16 21:40:05     0    2
 1    2018-12-16 21:45:05     0    2
 2    2018-12-16 21:50:05     0    2
 3    2018-12-16 21:55:06     1    2
 4    2018-12-16 22:00:05     1    2
 ...                  ...   ...  ...
 4331 2018-12-31 23:35:04     4    3
 4332 2018-12-31 23:40:05     4    3
 4333 2018-12-31 23:45:05     4    3
 4334 2018-12-31 23:50:25     4    3
 4335 2018-12-31 23:55:05     4    3
 
 [4336 rows x 3 columns],
                   datetime  wash  dry
 0      2019-01-01 00:00:05     4    3
 1      2019-01-01 00:05:27     4    3
 2      2019-01-01 00:10:05     4    3
 3      2019-01-01 00:15:05     4    3
 4      2019-01-01 00:20:26     4    3
 ...                    ...   ...  ...
 102113 2019-12-31 23:35:04     4    3
 102114 2019-12-31 23:40:06     4    3
 102115 2019-12-31 23:45:04     4    3
 102116 2019-12-31 23:50:04     4    3
 102117 2019-12-31 23:55:05     4    3
 
 [102118 rows x 3 columns],
                   datetime  w

In [5]:
def get_weekdays(data):
    return data[data.set_index('datetime').index.dayofweek < 5]

def get_weekends(data):
    return data[data.set_index('datetime').index.dayofweek >= 5]

print(get_weekdays(months[1]))
#print(get_weekends(months[1]))

                datetime  wash  dry
0    2019-01-01 00:00:05     4    3
1    2019-01-01 00:05:27     4    3
2    2019-01-01 00:10:05     4    3
3    2019-01-01 00:15:05     4    3
4    2019-01-01 00:20:26     4    3
...                  ...   ...  ...
8227 2019-01-31 23:35:05     0    3
8228 2019-01-31 23:40:05     0    3
8229 2019-01-31 23:45:05     2    3
8230 2019-01-31 23:50:05     2    3
8231 2019-01-31 23:55:05     2    3

[6463 rows x 3 columns]


In [6]:
def average_range(data):
    local_data=data.copy()
    local_data['datetime'] = [time.floor("5min").time() for time in local_data['datetime']]
    return local_data.set_index('datetime').groupby('datetime').mean().reset_index()

print(average_range(months[1]))

     datetime      wash       dry
0    00:00:00  3.551724  2.551724
1    00:05:00  3.482759  2.586207
2    00:10:00  3.310345  2.482759
3    00:15:00  3.310345  2.482759
4    00:20:00  3.250000  2.464286
..        ...       ...       ...
283  23:35:00  2.586207  2.103448
284  23:40:00  2.620690  2.275862
285  23:45:00  2.896552  2.310345
286  23:50:00  3.103448  2.310345
287  23:55:00  3.137931  2.448276

[288 rows x 3 columns]


In [7]:
def shift_time_forward(data):
    local_data=data.copy()
    local_data['datetime'] = [datetime_time((time.hour + 4)%24, time.minute) for time in local_data['datetime']]
    return local_data

def make_pretty(data_ref):
    data = data_ref.copy()
    data.names = ["Time", "Washers Available", "Dryers Available"]

def plot_data(data_ref, title, filename):
    data = data_ref.copy()
    data['int_index'] = range(len(data))
    #ticks=[time.time() for time in pd.date_range(start='1/1/2018 4:00', end='1/3/2018', freq='4H')]
    #tickss=[tick.strftime("%H:%M") for tick in ticks]
    ax = data.plot(x="datetime", y=["wash", "dry"], figsize=(12, 6), title=title)
    #ax.set_xticklabels(tickss)
    ax.legend(["Washers Available", "Dryers Available"])
    ax.set_xlabel("Time")
    fig = ax.get_figure()
    fig.savefig("plots/"+filename)
    plt.close(fig)


#plot_data(average_range(get_weekdays(get_by_months(raw_data)[1])), "Weekdays Jan 2019", "./test.png")

In [8]:
def process_range(data, label, filename):
    plot_data(average_range(get_weekdays(data)), "Weekdays " + label, f"{filename}_weekdays.png")
    plot_data(average_range(get_weekends(data)), "Weekend " + label, f"{filename}_weekends.png")

In [9]:
raw_data['datetime'].min()

Timestamp('2018-12-16 21:40:05')

In [10]:
months = get_by_months(raw_data)[1:]
years = get_by_year(raw_data)[1:]

In [11]:
#for month in months:
#    title = month['datetime'][0].strftime("%B %Y")
#    m = month['datetime'][0].month
#    y = month['datetime'][0].year
#    pref = str(y) + "." + str(m).zfill(2)
#    basename = f"{pref}"
#    process_range(month, title, basename)

In [12]:
for year in years:
    title = year['datetime'][0].strftime("%B %Y")
    y = year['datetime'][0].year
    try:
        process_range(year, title, str(y))
    except TypeError:
        print(year['datetime'].max())

In [13]:
process_range(shift_back(raw_data), "all time Average", "0.0")